<a href="https://colab.research.google.com/github/shakti1590/python_task/blob/main/scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from bs4 import BeautifulSoup
import pandas as pd
import requests

excel_file = 'data.xlsx'
data_frame = pd.read_excel(excel_file)
scraped_data = []
for index, row in data_frame.iterrows():
    url = f"https://www.amazon.{row['country']}/dp/{row['asin']}"

    try:
        response = requests.get(url)
        if response.status_code == 404:
            print(f"{url} not available")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        product_title = soup.find('span', {'id': 'productTitle'}).get_text().strip()
        product_image_url = soup.find('img', {'id': 'landingImage'})['src']
        product_price = soup.find('span', {'id': 'priceblock_ourprice'}).get_text().strip()
        product_details = []
        for li in soup.find_all('li', {'class': 'a-list-item'}):
            product_details.append(li.get_text().strip())
        product_data = {
            'url': url,
            'title': product_title,
            'image_url': product_image_url,
            'price': product_price,
            'details': product_details
        }

        scraped_data.append(product_data)
        print(f"Scraped data for {url}")

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        continue
with open('scraped_data.json', 'w') as json_file:
    json.dump(scraped_data, json_file, indent=4)
